In [1]:
import torch
import numpy as np
from scipy.spatial import distance
import os

import sys
sys.path.append('..')
import registry
import datafree
%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [2]:
!nvidia-smi

Sun Aug 21 02:19:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.60.02    Driver Version: 510.60.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:05:00.0 Off |                  N/A |
| 30%   33C    P0    94W / 350W |      0MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:09:00.0 Off |                  N/A |
| 30%   

In [3]:
distributed = False
gpu = 5
# gpu ='0,1'
batch_size = 128
workers = 8
# num_classes = 10
# num_classes = 100
num_classes = 200
def prepare_model(model):
    if not torch.cuda.is_available():
        print('using CPU, this will be slow')
        return model
    elif distributed:
        # For multiprocessing distributed, DistributedDataParallel constructor
        # should always set the single device scope, otherwise,
        # DistributedDataParallel will use all available devices.
        if gpu is not None:
#             torch.cuda.set_device(gpu)
            model.cuda()
            # When using a single GPU per process and per
            # DistributedDataParallel, we need to divide the batch size
            # ourselves based on the total number of GPUs we have
            batch_size = int(batch_size / 1)
            workers = int((workers + 1 - 1) / 1)
            model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[int(x) for x in gpu.split(',')])
            return model
        else:
            model.cuda()
            model = torch.nn.parallel.DistributedDataParallel(model)
            return model
    elif gpu is not None:
        torch.cuda.set_device(gpu)
        model = model.cuda(gpu)
        return model
    else:
        # DataParallel will divide and allocate batch_size to all available GPUs
        model = torch.nn.DataParallel(model).cuda()
        return model

In [22]:
from torchvision.datasets import CIFAR10,CIFAR100
import datafree
import registry
from torch import nn
student = registry.get_model('resnet18_imagenet', num_classes=num_classes)
teacher = registry.get_model('resnet34_imagenet', num_classes=num_classes, pretrained=True).eval()
# student = registry.get_model('wrn40_1', num_classes=num_classes)
# student= registry.get_model('wrn16_2', num_classes=num_classes)
# teacher = registry.get_model('wrn40_2', num_classes=num_classes)
normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['tiny_imagenet'])
# normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['cifar10'])
# normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['cifar100'])
student = prepare_model(student)

# teacher = teacher.to(gpu)
teacher.avgpool = nn.AdaptiveAvgPool2d(1)
num_ftrs = teacher.fc.in_features
teacher.fc = nn.Linear(num_ftrs, 200)
teacher.conv1 = nn.Conv2d(3,64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
teacher.maxpool = nn.Sequential()
teacher = prepare_model(teacher)
ckpt = torch.load('../checkpoints/scratch/tiny_imagenet_resnet34_imagenet.pth', map_location='cpu')
dict_ckpt = dict()
for k, v in ckpt['state_dict'].items():
    dict_ckpt['.'.join(k.split('.')[1:])] = v
teacher.load_state_dict(dict_ckpt)
print(ckpt['best_acc1'])
# teacher.load_state_dict(torch.load('../checkpoints/scratch/cifar10_resnet34.pth', map_location='cpu')['state_dict'])
# teacher.load_state_dict(torch.load('../checkpoints/scratch/cifar10_wrn40_2.pth', map_location='cpu')['state_dict'])
# teacher.load_state_dict(torch.load('../checkpoints/scratch/cifar100_wrn40_2.pth', map_location='cpu')['state_dict'])
# teacher.load_state_dict(torch.load('../checkpoints/scratch/cifar100_resnet34.pth', map_location='cpu')['state_dict'])
teacher.eval()


61.47


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Sequential()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1,

In [5]:
!ls /data/lijingru/DataFree/checkpoints/datafree-dafl/

cifar10-resnet34-resnet18--dafl_interval_a2.pth
cifar10-vgg11-resnet18--dafl_interval_2.pth
log-cifar10-resnet34-resnet18-dafl_interval_a2.txt
log-cifar10-vgg11-resnet18-dafl_interval_2.txt
synthetic-dafl_interval_2.png
synthetic-dafl_interval_a2.png


In [27]:
# generator = datafree.models.generator.LargeGenerator(nz=512, ngf=64, img_size=32, nc=3)
# generator = prepare_model(generator)
# ckpt = torch.load('/data/lijingru/CMI/checkpoints/datafree-cmi/cifar100-resnet34-resnet18.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-deepinv/cifar100-resnet34-resnet18--deepinv_100a.pth', map_location='cpu')
# ckpt = torch.load('/data/lijingru/CMI/checkpoints/datafree-dafl/cifar100-resnet34-resnet18.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-cmi/cifar10-resnet34-resnet18_adv_cmi.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-cudfkd/cifar100-resnet34-resnet18--cudfkd_L2_line33_e.pth', map_location='cpu')
ckpt = torch.load('../checkpoints/datafree-probkd/tiny_imagenet-resnet34_imagenet-resnet18_imagenet--probkd_L3_line7-R0.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-cudfkd/cifar10-wrn40_2-wrn16_2--cudfkd_L2_line93_agg_2.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-probkd/cifar10-resnet34-resnet18--probkd_L2_line66.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-deepinv/cifar10-resnet34-resnet18.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/scratch_i/cifar10_resnet18.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/scratch/cifar10_wrn16_2.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/scratch/cifar10_wrn40_1.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/scratch/cifar100_resnet18.pth', map_location='cpu')
# ckpt = torch.load('../checkpoints/datafree-cudfkd/cifar100-wrn40_2-wrn40_1--cudfkd_L2_line33_d.pth', map_location='cpu')
print(ckpt['best_acc1'])
dict_ckpt = dict()
for k, v in ckpt['state_dict'].items():
    dict_ckpt['.'.join(k.split('.')[1:])] = v
# G_ckpt = ckpt['G_0']

# student = prepare_model(student)
# student.avgpool = nn.AdaptiveAvgPool2d(1)
# num_ftrs = student.fc.in_features
# student.fc = nn.Linear(num_ftrs, 200)
# student.conv1 = nn.Conv2d(3,64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
# student.maxpool = nn.Sequential()
student.load_state_dict(dict_ckpt)
student.eval()


43.419999999999995


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [24]:
import torchvision.transforms as T
from registry import NORMALIZE_DICT
from torchvision import datasets
# normalizer = datafree.utils.Normalizer(**registry.NORMALIZE_DICT['cifar10'])
# val_transform = T.Compose([
#     #T.Resize((224, 224), Image.BICUBIC),
#     T.ToTensor(),
#     T.Normalize( **NORMALIZE_DICT['tiny_imagenet'] ),
# ])
val_transform = T.Compose([
    T.ToTensor(),
    T.Normalize( **NORMALIZE_DICT['tiny_imagenet'] )]
) 

# val_dst = datasets.CIFAR10('/data/lijingru/cifar10/', train=False, download=True, transform=val_transform)
# val_dst = datasets.CIFAR100('/data/lijingru/cifar100/', train=False, download=True, transform=val_transform)
val_dst = datasets.ImageFolder(os.path.join('/data/lijingru/timagenet/tiny-imagenet-200', 'val_split'), transform=val_transform)

val_loader = torch.utils.data.DataLoader(val_dst, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

In [25]:
import tqdm
probs_s = []
probs_t = []
ys = []
for x, y in tqdm.tqdm(val_loader, desc='calculate_agg'):
    s_out = student(x.to(gpu))
    t_out = teacher(x.to(gpu))
    print(np.argmax(prob_t, 1), y.numpy())
#     print(torch.sum(s_out.argmax(1).detach().cpu() == y) / 128)
    prob_s = torch.softmax(s_out.detach(), 1).cpu().numpy()
    probs_s.append(prob_s)
    prob_t = torch.softmax(t_out.detach(), 1).cpu().numpy()
    probs_t.append(prob_t)
    ys.append(y.numpy())

probs_s = np.concatenate(probs_s, 0)
probs_t = np.concatenate(probs_t, 0)
ys = np.concatenate(ys, 0)

calculate_agg:   3%|▎         | 4/157 [00:00<00:18,  8.49it/s]

[189 187 185 199 131 185 189  38 189 110  38 130 199 176 168  15] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[  0   0   0   0 196   0   0   0   0   0  38  38   0 185  13  44  36   0
   0 176 102 115  36   0   0 151   0   0   0   0   0   0  38   0  38 161
   0  32   0   0   0   0   0   0 102 102  83   0   0  36 102 176 176 113
 141  44 176 176  38 176 176 113 113 176] [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[ 75  38  44  17  38  38 176 131 154  38  44 174 176 176   1 176 113 176
   8 176 176 113  44 176 176  44 176  39 168 156  64 156 196 176  44 176
  44  44 139  10 122 176 119   2   2  12  84 162 176   3   3  12  84  44
 196   5  92  92  44 185 102   2  39 122] [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
[176 152   2 199 19

calculate_agg:   7%|▋         | 11/157 [00:00<00:07, 18.90it/s]

[168 161  88   6 150   6  95  66 122   6 123   6 176   6  59  84 185 168
   6  12   6 199 111  66 164 102  16  12 106  16  88   7  17  74 156  10
  53 196   7 144 101  10  44   8  13  41  39   8   3  41   7 122 112 196
 139 132  41  41   7 176 156  76  88 102] [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[  7   6  38 115  75   1 176   7 149  13 176 134  13 161  44 122   9  38
  44 168 156  17   8 122   8 155   8 156 168  96 176   8   8   8   8   8
 151  38  38 122  36 168 101 168 111   8   8  44 122   8 159   8   8 174
   8 135   8   8   8 168   8   8 168   8] [ 8  8  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9 10 10 10 10 10 10 10 10 10 10 10 10]
[  8  64   9 156   9  41 130 176  37 122 149 176 156   9  41 159   3 176
 102   8 176 119 176  10 176 176 162  75  38 131  75  29  36  75 176  41
 122 

calculate_agg:  12%|█▏        | 19/157 [00:01<00:05, 24.86it/s]

...

calculate_agg:  96%|█████████▌| 151/157 [00:05<00:00, 30.48it/s]

[131 185 185  16 185 185 113  67  75 185 185 185 131 185 188 185 113 102
 185 130  38  75  38  38  36 113  38 188 176 187  38 179 186 186  38 186
 186  13  38 180  38 101  38  38 151 188 186 130 113  38  38  38 186 196
 186  38  38 168 102 113 186 187 186 186] [186 186 186 186 186 186 187 187 187 187 187 187 187 187 187 187 187 187
 187 187 187 187 187 187 187 187 187 187 187 187 187 187 187 187 187 187
 187 187 187 187 187 187 187 187 187 187 187 187 187 187 187 187 187 187
 187 187 188 188 188 188 188 188 188 188]
[196  38 188  38   6 187  92 186 185  38 113 130 187  38 187  92  38  38
 196 187 187 187  38  83 187 184 187  73 187 187 187 186 187 196 187 187
 187 187 185 196 186  38 187 130 102 186  38 186 187 113 187  38 187 156
 102 188  59 188 186 163 156 188 188 188] [188 188 188 188 188 188 188 188 188 188 188 188 188 188 188 188 188 188
 188 188 188 188 188 188 188 188 188 188 188 188 188 188 188 188 188 188
 188 188 188 188 188 188 189 189 189 189 189 189 189 189 189 189 189 18

calculate_agg:  99%|█████████▊| 155/157 [00:05<00:00, 30.41it/s]

[194 194 194 176 102 194 168 195  75  66 168 194 101  76 101 196 129 194
 162  22 148 122 132 195  60 197 195 102 197  41 162  96  44 126 195  29
 176  12  66 198  79 195 195 195 197 196  31  12 162 119  66 129  75 194
  66  96 194 102  64  66 195  12 176  38] [195 195 195 195 195 195 195 195 196 196 196 196 196 196 196 196 196 196
 196 196 196 196 196 196 196 196 196 196 196 196 196 196 196 196 196 196
 196 196 196 196 196 196 196 196 196 196 196 196 196 196 196 196 196 196
 196 196 196 196 197 197 197 197 197 197]
[195  95  12 195 195 176 195 194 196  23 196 196 176 196 196 196 176 196
 176 196 196 196 196 183  46 196 196 176 196  39 196 196 196 183 196 168
  14 196 176 196  14 196 196  13  72  13 196 196 196  38 196 196  46 196
  13 113  46  14 197 176 197 102 198 122] [197 197 197 197 197 197 197 197 197 197 197 197 197 197 197 197 197 197
 197 197 197 197 197 197 197 197 197 197 197 197 197 197 197 197 197 197
 197 197 197 197 197 197 197 197 198 198 198 198 198 198 198 198 198 19

calculate_agg: 100%|██████████| 157/157 [00:05<00:00, 27.28it/s]


In [26]:
if not os.path.exists('prob_loyalty_distribution/'):
    os.mkdir('prob_loyalty_distribution/')
# print(y)
print(np.sum(np.argmax(probs_s, 1) == ys) / len(val_dst))
print(np.sum(np.argmax(probs_t, 1) == ys) / len(val_dst))
print(np.sum(np.argmax(probs_t, 1) == np.argmax(probs_s, 1)) / len(val_dst))
dist = distance.jensenshannon(probs_s.T, probs_t.T)
dist[np.isnan(dist)] = 0 if np.sum(np.abs(probs_s[np.where(np.isnan(dist))] - probs_t[np.where(np.isnan(dist))])) < 1e-6 else 1
prob_loyalty = 1 - np.sqrt(dist)
print(np.nanmean(prob_loyalty))
# print(dist[1773])

0.1192
0.2485
0.291
0.25757793


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
fig = sns.displot(prob_loyalty, kind='hist', color=sns.xkcd_rgb["pale red"])
# fig.set_xlim(0, 1)
fig.set_xlabels('probability loyalty', fontsize=14)
fig.set_ylabels('Number of Images', fontsize=14)
# plt.savefig('prob_loyalty_distribution/prob_loyalty_deepinv.pdf')
# x_ticks = np.arange(0, 1.01, 0.2)
# y_ticks = np.arange(0, 700, 100)
# fig.set_xticklabels(x_ticks, fontsize=12)
# fig.set_yticklabels(y_ticks, fontsize=12)


<Figure size 360x360 with 1 Axes>

### Other visualization Results
The items for further visualization:
- Difficulty visualization at different timestage.
- Ablation study of $k_{begin}$ and $k_{end}$.
- Ablation study of $\alpha_{adv}$.
- Error bar of CuDFKD, CMI, ADI and DAFL.

In [26]:
x = np.array(['1/5', '1/4', '1/3'])
y = np.array(['2/3', '3/4', '4/5'])
res2res_acc = np.array([[71.00, 71.22, 71.12], [70.81, 70.79, 70.68], [70.41, 70.52, 70.54]])
res2res_agr = np.array([[85.88, 85.85, 85.64], [85.40, 85.14, 85.58], [83.77, 83.88, 83.67]])
res2res_loyalty = np.array([[0.6659, 0.6777, 0.6751], [0.6684, 0.6624, 0.6682], [0.6469, 0.6500, 0.6467]])

In [14]:
if not os.path.exists('heatmaps/'):
    os.mkdir('heatmaps/')
fig1 = sns.heatmap(res2res_acc, xticklabels=x, yticklabels=y, annot=True, fmt='.2f', cmap='RdBu_r')
plt.savefig('heatmaps/vgg_acc.pdf')

<Figure size 432x288 with 2 Axes>

In [15]:
fig2 = sns.heatmap(res2res_agr, xticklabels=x, yticklabels=y, annot=True, fmt='.2f', cmap='RdBu_r')
plt.savefig('heatmaps/vgg_agg.pdf')

<Figure size 432x288 with 2 Axes>

In [16]:
fig3 = sns.heatmap(res2res_loyalty, xticklabels=x, yticklabels=y, annot=True, fmt='.4f', cmap='RdBu_r')
plt.savefig('heatmaps/vgg_loyalty.pdf')

<Figure size 432x288 with 2 Axes>

In [17]:
vgg2res_acc = np.array([[64.74, 64.65, 64.77], [64.66, 64.66, 64.10], [63.95, 64.00, 63.81]])
vgg2res_agr = np.array([[68.91, 68.88, 68.93], [68.80, 68.85, 68.54], [68.58, 68.19, 68.31]])
vgg2res_loyalty = np.array([[0.5067, 0.5058, 0.5072], [0.5059, 0.5054, 0.5069], [0.5001, 0.4961, 0.5021]])

In [18]:
sns.heatmap(vgg2res_acc, xticklabels=x, yticklabels=y, annot=True, fmt='.2f', cmap='RdBu_r')
plt.savefig('heatmaps/wrn_acc.pdf')

<Figure size 432x288 with 2 Axes>

In [19]:
sns.heatmap(vgg2res_agr, xticklabels=x, yticklabels=y, annot=True, fmt='.2f', cmap='RdBu_r')
plt.savefig('heatmaps/wrn_agg.pdf')

<Figure size 432x288 with 2 Axes>

In [20]:
sns.heatmap(vgg2res_loyalty, xticklabels=x, yticklabels=y, annot=True, fmt='.4f', cmap='RdBu_r')
plt.savefig('heatmaps/wrn_loyalty.pdf')

<Figure size 432x288 with 2 Axes>

In [12]:
ckpt = torch.load('../checkpoints/datafree-cudfkd/cifar100-wrn40_2-wrn40_1--cudfkd_L2_line33_d.pth', map_location='cpu')
student.load_state_dict(ckpt['state_dict'])
student.eval()
g = datafree.models.generator.DCGAN_Generator_CIFAR10(nz=512, ngf=64, nc=3, img_size=32, d=2, cond=False)
g = prepare_model(g)
g.load_state_dict(ckpt['G_0'])
g.eval()

DCGAN_Generator_CIFAR10(
  (project): Sequential(
    (0): Flatten()
    (1): Linear(in_features=512, out_features=16384, bias=True)
  )
  (main): Sequential(
    (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode=nearest)
    (2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Sigmoid()
  )
)

In [13]:
from torchvision.utils import make_grid,save_image
diff = []
now_epoch = 300
kl_metric = datafree.criterions.KLDiv(T=2., reduction='none')
with torch.no_grad():
    for i in range(20):
        z = torch.randn(batch_size, 512).to(gpu)
        x = g(z)
        t_out = teacher(normalizer(x))
        s_out = student(normalizer(x))
        kl_diff = kl_metric(t_out, s_out).sum(1).detach().cpu().numpy()
        diff.append(kl_diff)
    
diff = np.concatenate(diff)
print(diff)
with torch.no_grad():
    z = torch.randn(1024, 512).to(gpu)
    img = make_grid(x, nrow=16)
    t_out = teacher(normalizer(x))
    pesudo_label = t_out.argmax(1).cpu().numpy()
    print(pesudo_label)

#     save_image(img, 'difficulty/epoch_{}.jpg'.format(now_epoch))

[1.6653383  2.05072    1.3142852  ... 1.1158123  0.25667655 2.7491198 ]
[41 44 80 ... 86 96 67]


In [19]:
xs = []
for i in range(num_classes):
#     print(i)
    xs.append(x[pesudo_label == i].cpu())
    print(i, xs[i].shape)
    img = make_grid(xs[i], nrow=1)
    save_image(img, 'difficulty/img_epoch_{}_{}.jpg'.format(now_epoch, i))

0 torch.Size([8, 3, 32, 32])
1 torch.Size([13, 3, 32, 32])
2 torch.Size([15, 3, 32, 32])
3 torch.Size([11, 3, 32, 32])
4 torch.Size([9, 3, 32, 32])
5 torch.Size([11, 3, 32, 32])
6 torch.Size([12, 3, 32, 32])
7 torch.Size([9, 3, 32, 32])
8 torch.Size([11, 3, 32, 32])
9 torch.Size([5, 3, 32, 32])
10 torch.Size([12, 3, 32, 32])
11 torch.Size([6, 3, 32, 32])
12 torch.Size([8, 3, 32, 32])
13 torch.Size([11, 3, 32, 32])
14 torch.Size([11, 3, 32, 32])
15 torch.Size([10, 3, 32, 32])
16 torch.Size([9, 3, 32, 32])
17 torch.Size([8, 3, 32, 32])
18 torch.Size([9, 3, 32, 32])
19 torch.Size([6, 3, 32, 32])
20 torch.Size([9, 3, 32, 32])
21 torch.Size([9, 3, 32, 32])
22 torch.Size([13, 3, 32, 32])
23 torch.Size([8, 3, 32, 32])
24 torch.Size([4, 3, 32, 32])
25 torch.Size([6, 3, 32, 32])
26 torch.Size([18, 3, 32, 32])
27 torch.Size([10, 3, 32, 32])
28 torch.Size([11, 3, 32, 32])
29 torch.Size([10, 3, 32, 32])
30 torch.Size([12, 3, 32, 32])
31 torch.Size([11, 3, 32, 32])
32 torch.Size([6, 3, 32, 32])
33 

In [21]:
sns.displot(diff)

<Figure size 360x360 with 1 Axes>

In [21]:
wrn2wrn_acc = np.array([[75.24, 75.16, 75.31], [75.11, 75.21, 75.22], [75.01, 74.89, 75.01]])
wrn2wrn_agr = np.array([[85.26, 86.06, 86.11], [85.40, 85.14, 85.58], [85.38, 85.53, 85.57]])
wrn2wrn_loyalty = np.array([[0.6472, 0.6476, 0.6476], [0.6433, 0.6437, 0.6447], [0.6393, 0.6403, 0.6407]])


In [22]:
sns.heatmap(wrn2wrn_acc, xticklabels=x, yticklabels=y, annot=True, fmt='.2f', cmap='RdBu_r')
plt.savefig('heatmaps/res_acc.pdf')

<Figure size 432x288 with 2 Axes>

In [23]:
sns.heatmap(wrn2wrn_agr, xticklabels=x, yticklabels=y, annot=True, fmt='.2f', cmap='RdBu_r')
plt.savefig('heatmaps/res_agg.pdf')

<Figure size 432x288 with 2 Axes>

In [25]:
sns.heatmap(wrn2wrn_loyalty, xticklabels=x, yticklabels=y, annot=True, fmt='.4f', cmap='RdBu_r')
plt.savefig('heatmaps/res_loyalty.pdf')

<Figure size 432x288 with 2 Axes>

In [46]:
import pandas as pd
grad_range = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4])
adv_acc = np.array([75.10, 75.31, 75.50, 75.50, 75.51, 75.60, 75.73, 75.67])
adv_agg = np.array([85.52, 86.11, 86.22, 86.52, 86.30, 86.52, 86.32, 86.11])
adv_loyalty = np.array([0.6412, 0.6476, 0.6528, 0.6568, 0.6571, 0.6576, 0.6575, 0.6572])
if not os.path.exists('grad_adv/'):
    os.mkdir('grad_adv/')

res_dict = {
    'grad_of_adv': grad_range,
    'acc@1': adv_acc,
    'agree@1': adv_agg,
    'loyalty': adv_loyalty
}

final_df = pd.DataFrame(res_dict)
final_df

,grad_of_adv,acc@1,agree@1,loyalty
0,0.5,75.10,85.52,0.6412
1,1.0,75.31,86.11,0.6476
2,1.5,75.50,86.22,0.6528
3,2.0,75.50,86.52,0.6568
4,2.5,75.51,86.30,0.6571
5,3.0,75.60,86.52,0.6576
6,3.5,75.73,86.32,0.6575
7,4.0,75.67,86.11,0.6572


In [21]:
# now_epoch = 300
global_iter = now_epoch * 400
lamda = datafree.datasets.utils.lambda_scheduler(1.0, global_iter, alpha=0.00002)
g,v = datafree.datasets.utils.curr_v(l=torch.FloatTensor(diff), lamda=lamda, spl_type='log')

In [22]:
if not os.path.exists('difficulty/'):
    os.mkdir('difficulty/')
diffs = dict()
diffs['epoch@{}'.format(now_epoch)] = v.numpy() * diff
# sns.displot(v.numpy() * diff, kind='kde')
# plt.savefig('difficulty/difficulty_at_{}.pdf'.format(now_epoch))

In [23]:
ckpt = torch.load('../checkpoints/datafree-cudfkd/cifar100-wrn40_2-wrn40_1-10.pth', map_location='cpu')
student.load_state_dict(ckpt['state_dict'])
student.eval()
g = datafree.models.generator.DCGAN_Generator_CIFAR10(nz=512, ngf=64, nc=3, img_size=32, d=2, cond=False)
g = prepare_model(g)
g.load_state_dict(ckpt['G_0'])
g.eval()

DCGAN_Generator_CIFAR10(
  (project): Sequential(
    (0): Flatten()
    (1): Linear(in_features=512, out_features=16384, bias=True)
  )
  (main): Sequential(
    (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode=nearest)
    (2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode=nearest)
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Sigmoid()
  )
)

In [24]:
diff = []
now_epoch = 10
kl_metric = datafree.criterions.KLDiv(T=2., reduction='none')
with torch.no_grad():
    for i in range(20):
        z = torch.randn(batch_size, 512).to(gpu)
        x = g(z)
        t_out = teacher(normalizer(x))
        s_out = student(normalizer(x))
        kl_diff = kl_metric(t_out, s_out).sum(1).detach().cpu().numpy()
        diff.append(kl_diff)
    
diff = np.concatenate(diff)
print(diff)
with torch.no_grad():
    z = torch.randn(1024, 512).to(gpu)
    x = g(z)
    img = make_grid(x, nrow=16)
    t_out = teacher(normalizer(x))
    pesudo_label = t_out.argmax(1).cpu().numpy()
    print(pesudo_label)
    save_image(img, 'difficulty/epoch_{}.jpg'.format(now_epoch))

[0.44048202 0.7211295  0.7666422  ... 0.34204212 0.96407497 0.6640957 ]
[42 24 45 ... 99 60 87]


In [26]:
xs = []
for i in range(num_classes):
#     print(i)
    xs.append(x[pesudo_label == i].cpu())
    print(i, xs[i].shape)
    if xs[i].shape[0] > 0:
        img = make_grid(xs[i], nrow=1)
        save_image(img, 'difficulty/img_epoch_{}_{}.jpg'.format(now_epoch, i))

0 torch.Size([3, 3, 32, 32])
1 torch.Size([25, 3, 32, 32])
2 torch.Size([9, 3, 32, 32])
3 torch.Size([15, 3, 32, 32])
4 torch.Size([18, 3, 32, 32])
5 torch.Size([24, 3, 32, 32])
6 torch.Size([10, 3, 32, 32])
7 torch.Size([20, 3, 32, 32])
8 torch.Size([15, 3, 32, 32])
9 torch.Size([5, 3, 32, 32])
10 torch.Size([15, 3, 32, 32])
11 torch.Size([8, 3, 32, 32])
12 torch.Size([3, 3, 32, 32])
13 torch.Size([8, 3, 32, 32])
14 torch.Size([9, 3, 32, 32])
15 torch.Size([11, 3, 32, 32])
16 torch.Size([14, 3, 32, 32])
17 torch.Size([6, 3, 32, 32])
18 torch.Size([4, 3, 32, 32])
19 torch.Size([12, 3, 32, 32])
20 torch.Size([2, 3, 32, 32])
21 torch.Size([2, 3, 32, 32])
22 torch.Size([6, 3, 32, 32])
23 torch.Size([22, 3, 32, 32])
24 torch.Size([11, 3, 32, 32])
25 torch.Size([4, 3, 32, 32])
26 torch.Size([22, 3, 32, 32])
27 torch.Size([6, 3, 32, 32])
28 torch.Size([15, 3, 32, 32])
29 torch.Size([10, 3, 32, 32])
30 torch.Size([19, 3, 32, 32])
31 torch.Size([10, 3, 32, 32])
32 torch.Size([8, 3, 32, 32])
33

In [27]:
# now_epoch = 10
global_iter = now_epoch * 400
lamda = datafree.datasets.utils.lambda_scheduler(1.0, global_iter, alpha=0.00002)
g,v = datafree.datasets.utils.curr_v(l=torch.FloatTensor(diff), lamda=lamda, spl_type='log')

In [28]:
ax = sns.displot(diffs, kind='kde')
ax.set_xlabels('Difficulty', fontsize=12)
plt.savefig('difficulty/difficulty_wrn402_wrn162_cifar100.pdf')

<Figure size 455x360 with 1 Axes>

In [74]:
res_acc_dict = {
    'method': ['cudfkd', 'cudfkd', 'cudfkd', 'teacher', 'adi', 'adi', 'adi', 'dafl', 'dafl', 'dafl', 'cmi', 'cmi', 'cmi'],
    'acc@1': [95.24, 95.28, 95.15, 95.70],
    'agree@1': [98.20, 98.19, 98.09, 100.00],
    'prob_loyalty': [0.8909, 0.8903, 0.8898, 1.00],
}

In [38]:
fig, ax1 = plt.subplots()
width = 0.4
x1_list = []
x2_list = []
for i in range(adv_acc.shape[0]):
    x1_list.append(i)
    x2_list.append(i + width)

# b1 = sns.lineplot(data=final_df, x="grad_of_adv", y="acc@1",color=sns.xkcd_rgb["pale red"] )  
b1, = plt.plot(final_df["grad_of_adv"], final_df["acc@1"], '-ro', label='Accuracy@1')

# b1 = ax1.bar(x1_list, adv_acc, width=width, label='Acc@1', color=sns.xkcd_rgb["pale red"], tick_label=grad_range)
ax2 = ax1.twinx()
b2, = plt.plot(final_df["grad_of_adv"], final_df["agree@1"], '-bo', label='Agreement@1')
# plt.grid('off')
# b2 = sns.lineplot(data=final_df, x="grad_of_adv", y="agree@1", color=sns.xkcd_rgb["denim blue"])
# b2 = ax2.bar(x2_list, adv_agg, width=width, label='Agree@1', color=sns.xkcd_rgb["denim blue"], tick_label=grad_range)
ax1.set_xlabel('grad of adv', fontsize=12)
ax1.set_ylabel('Accuracy@1', fontsize=12)
ax2.set_ylabel('Agreement@1', fontsize=12)
ax1.set_ylim(74, 77)
ax2.set_ylim(85, 87)
axs = [b1, b2]
lbs = [ax.get_label() for ax in axs]
ax1.legend(axs, lbs)
plt.savefig('grad_adv/acc_agg.pdf')
# plt.legend(['a', 'b'])
# plt.grid('off')

<Figure size 432x288 with 2 Axes>

In [39]:
plt.plot(final_df["grad_of_adv"], final_df["loyalty"], '-go', label="prob_loyalty")
plt.ylim(0.64, 0.66)
x_ticks = final_df["grad_of_adv"]
y_ticks = np.arange(0.64, 0.66, 0.005)
plt.xlabel('grad of adv', fontsize=14)
plt.ylabel('probability loyalty', fontsize=14)
plt.xticks(x_ticks, fontsize=12)
plt.yticks(y_ticks, fontsize=12)
plt.savefig('grad_adv/prob_loyalty.pdf')

<Figure size 432x288 with 1 Axes>

In [40]:
curriculum_dict={
    'method': ['teacher', 'None', 'hard', 'soft', 'log','teacher', 'None', 'hard', 'soft', 'log','teacher', 'None', 'hard', 'soft', 'log'],
    'accuracy': [95.70, 93.24, 94.97, 95.01, 95.28, 100.00, 94.00, 97.71, 97.74, 98.13, 1.0000, 0.8202, 0.8813, 0.8786, 0.8909],
    'metric': ['Acc@1','Acc@1','Acc@1','Acc@1','Acc@1','Agree@1','Agree@1','Agree@1','Agree@1','Agree@1','prob loyalty','prob loyalty','prob loyalty','prob loyalty','prob loyalty']
}
curr_df = pd.DataFrame(curriculum_dict)
curr_df
if not os.path.exists('curr_strategy/'):
    os.mkdir('curr_strategy/')

In [53]:
fig, ax1 = plt.subplots()
width = 0.4
x1_list = []
x2_list = []
for i in range(5):
    x1_list.append(i)
    x2_list.append(i + width)

# b1 = sns.lineplot(data=final_df, x="grad_of_adv", y="acc@1",color=sns.xkcd_rgb["pale red"] )  
# b1, = plt.plot(final_df["grad_of_adv"], final_df["acc@1"], '-ro', label='Accuracy@1')
# b1 = sns.barplot(data=curriculum_dict, x='method', y='accuracy', color=sns.xkcd_rgb["pale red"])
b1 = ax1.bar(x1_list, curr_df['accuracy'][:5], width=width, label='Acc@1', color=sns.xkcd_rgb["pale red"], tick_label=curriculum_dict['method'][:5])
ax2 = ax1.twinx()
# b2, = plt.plot(final_df["grad_of_adv"], final_df["agree@1"], '-bo', label='Agreement@1')
# plt.grid('off')
# b2 = sns.lineplot(data=final_df, x="grad_of_adv", y="agree@1", color=sns.xkcd_rgb["denim blue"])
b2 = ax2.bar(x2_list, curr_df['accuracy'][5:10], width=width, label='Agree@1', color=sns.xkcd_rgb["denim blue"], tick_label=curriculum_dict['method'][:5])
# ax2 = ax1.twinx()
# b2 = sns.barplot(data=curriculum_dict, x='method', y='Agree@1', color=sns.xkcd_rgb["denim blue"])
ax1.set_xlabel('Curriculum strategy', fontsize=12)
ax1.set_ylabel('Accuracy@1', fontsize=12)
ax2.set_ylabel('Agreement@1', fontsize=12)
ax1.set_ylim(93, 96)
ax2.set_ylim(92, 101)
# ax2.set_yticklabels(np.arange(92, 101, 2))
axs = [b1, b2]
lbs = [ax.get_label() for ax in axs]
# ax1.legend()
ax1.legend(axs, lbs)
plt.savefig('curr_strategy/acc_agg.pdf')

<Figure size 432x288 with 2 Axes>

In [63]:
plt.bar(x1_list, curr_df['accuracy'][-5:], width=width, label='prob_loyalty', color=sns.xkcd_rgb["dark green"], tick_label=['teacher', 'None', 'hard', 'soft', 'log'])
plt.ylim(0.8, 1.01)
# x_ticks = final_df["grad_of_adv"]
# x_ticks = ['teacher', 'None', 'hard', 'soft', 'log']
y_ticks = np.arange(0.8, 1.01, 0.05)
plt.xlabel('Curriculum Strategy', fontsize=14)
plt.ylabel('probability loyalty', fontsize=14)
plt.xticks(x_ticks, fontsize=12)
plt.yticks(y_ticks, fontsize=12)
plt.legend()
plt.savefig('curr_strategy/prob_loyalty.pdf')

ConversionError: Failed to convert value(s) to axis units: ['teacher', 'None', 'hard', 'soft', 'log']

<Figure size 432x288 with 1 Axes>

In [47]:
if not os.path.exists('T/'):
    os.mkdir('T/')
T_dict = {
    'T': [4, 5, 8, 10, 12, 15, 18, 20],
    'accuracy@1': [94.96, 95.01,94.94,94.93,94.97, 95.03, 94.94, 95.28],
    'agreement@1': [97.86, 97.64, 97.79, 97.63, 97.60, 97.69, 97.74, 98.20],
    'prob_loyalty': [0.8777, 0.8774, 0.8807, 0.8739, 0.8771, 0.8801, 0.8793, 0.8909]
    
}


In [50]:
fig, ax1 = plt.subplots()
width = 0.4
x1_list = []
x2_list = []
for i in range(adv_acc.shape[0]):
    x1_list.append(i)
    x2_list.append(i + width)

# b1 = sns.lineplot(data=final_df, x="grad_of_adv", y="acc@1",color=sns.xkcd_rgb["pale red"] )  
b1, = plt.plot(T_dict["T"], T_dict['accuracy@1'], '-ro', label='Accuracy@1')

# b1 = ax1.bar(x1_list, adv_acc, width=width, label='Acc@1', color=sns.xkcd_rgb["pale red"], tick_label=grad_range)
ax2 = ax1.twinx()
b2, = plt.plot(T_dict["T"], T_dict['agreement@1'], '-bo', label='Agreement@1')
# plt.grid('off')
# b2 = sns.lineplot(data=final_df, x="grad_of_adv", y="agree@1", color=sns.xkcd_rgb["denim blue"])
# b2 = ax2.bar(x2_list, adv_agg, width=width, label='Agree@1', color=sns.xkcd_rgb["denim blue"], tick_label=grad_range)
ax1.set_xlabel('T', fontsize=12)
ax1.set_ylabel('Accuracy@1', fontsize=12)
ax2.set_ylabel('Agreement@1', fontsize=12)
ax1.set_ylim(94, 95.5)
ax2.set_ylim(97, 99)
axs = [b1, b2]
lbs = [ax.get_label() for ax in axs]
ax1.legend(axs, lbs)
plt.savefig('T/acc_agg.pdf')

<Figure size 432x288 with 2 Axes>

In [54]:
plt.plot(T_dict["T"], T_dict["prob_loyalty"], '-go', label="prob_loyalty")
plt.ylim(0.87, 0.9)
x_ticks = np.arange(4, 21, 4)
y_ticks = np.arange(0.87, 0.9, 0.005)
plt.xlabel('T', fontsize=14)
plt.ylabel('probability loyalty', fontsize=14)
plt.xticks(x_ticks, fontsize=12)
plt.yticks(y_ticks, fontsize=12)
plt.savefig('T/prob_loyalty.pdf')

<Figure size 432x288 with 1 Axes>

In [50]:
if not os.path.exists('lambda_0/'):
    os.mkdir('lambda_0/')
lambda_0_dict = {
    'lambda_0': [2, 2.2, 2.5, 2.8],
    'accuracy@1': [93.34,93.24,93.17,93.10],
    'agreement@1': [94.15, 94.02, 93.92, 93.97],
    'loyalty': [0.8183, 0.8176, 0.8173, 0.8170],
    'curriculum': ['logarithm', 'logarithm', 'logarithm', 'logarithm']
}



In [52]:
fig, ax1 = plt.subplots()
# width = 0.4
# x1_list = []
# x2_list = []
# for i in range(adv_acc.shape[0]):
#     x1_list.append(i)
#     x2_list.append(i + width)

# b1 = sns.lineplot(data=final_df, x="grad_of_adv", y="acc@1",color=sns.xkcd_rgb["pale red"] )  
b1, = plt.plot(lambda_0_dict["lambda_0"], lambda_0_dict['accuracy@1'], '-ro', label='Accuracy@1')

# b1 = ax1.bar(x1_list, adv_acc, width=width, label='Acc@1', color=sns.xkcd_rgb["pale red"], tick_label=grad_range)
ax2 = ax1.twinx()
b2, = plt.plot(lambda_0_dict["lambda_0"], lambda_0_dict['agreement@1'], '-bo', label='Agreement@1')
# plt.grid('off')
# b2 = sns.lineplot(data=final_df, x="grad_of_adv", y="agree@1", color=sns.xkcd_rgb["denim blue"])
# b2 = ax2.bar(x2_list, adv_agg, width=width, label='Agree@1', color=sns.xkcd_rgb["denim blue"], tick_label=grad_range)
ax1.set_xlabel('lambda_0', fontsize=12)
ax1.set_ylabel('Accuracy@1', fontsize=12)
ax2.set_ylabel('Agreement@1', fontsize=12)
ax1.set_ylim(93, 93.5)
ax2.set_ylim(93.8, 94.2)
axs = [b1, b2]
lbs = [ax.get_label() for ax in axs]
ax1.legend(axs, lbs)
plt.savefig('lambda_0/acc_agg.pdf')

<Figure size 432x288 with 2 Axes>

In [53]:
plt.plot(lambda_0_dict["lambda_0"], lambda_0_dict["loyalty"], '-go', label="prob_loyalty")
plt.ylim(0.815, 0.82)
x_ticks = np.arange(2, 3, 0.2)
y_ticks = np.arange(0.815, 0.82, 0.001)
plt.xlabel('lambda_0', fontsize=14)
plt.ylabel('probability loyalty', fontsize=14)
plt.xticks(x_ticks, fontsize=12)
plt.yticks(y_ticks, fontsize=12)
plt.savefig('lambda_0/prob_loyalty.pdf')

<Figure size 432x288 with 1 Axes>

In [36]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'pdf'
res_dict_below = {
    'method': ['teacher', 'teacher', 'cudfkd', 'cudfkd', 'cmi', 'cmi', 'dafl', 'dafl', 'adi', 'adi', 'adi', 'adi', 'cudfkd', 'cudfkd', 'dafl', 'dafl', 'cmi', 'cmi', 'adi', 'adi', 'cmi', 'cmi', 'dafl', 'dafl', 'cudfkd', 'cudfkd'],
    'acc@1': [95.70, 94.07, 95.28, 93.24, 94.84, 92.36, 91.45 , 81.66, 93.26, 90.38, 90.36, 90.17, 95.24, 95.15, 82.89, 81.86, 91.95, 91.85, 94.56, 94.63, 93.18, 93.36 , 89.41, 89.04, 93.38, 93.30],
    'type': ['clean', 'noisy', 'clean', 'noisy','clean', 'noisy','clean', 'noisy','clean', 'noisy', 'noisy', 'noisy', 'clean', 'clean', 'noisy', 'noisy', 'noisy', 'noisy', 'clean', 'clean', 'clean', 'clean', 'clean', 'clean', 'noisy', 'noisy'],
    'agree@1':[100.00, 100.00, 98.19, 97.25, 96.46, 93.13, 93.20, 83.85, 93.45, 92.88, 92.90, 92.92 , 98.20, 98.09, 85.03, 83.55, 94.98, 95.10, 96.96, 96.85, 95.23, 95.34, 91.06, 90.69, 97.69, 97.27],
    'loyalty':[1.0000, 1.0000, 0.8909, 0.9002, 0.8747, 0.8049, 0.7686, 0.6521, 0.8078, 0.8247, 0.8235, 0.8245, 0.8915, 0.8897, 0.6803, 0.6527, 0.8701, 0.8711, 0.8849, 0.8833, 0.8501,0.8499, 0.7298, 0.7336, 0.9063, 0.8998],
}

noisy_df = pd.DataFrame(data=res_dict_below)
noisy_df

,method,acc@1,type,agree@1,loyalty
0,teacher,95.70,clean,100.00,1.0000
1,teacher,94.07,noisy,100.00,1.0000
2,cudfkd,95.28,clean,98.19,0.8909
3,cudfkd,93.24,noisy,97.25,0.9002
4,cmi,94.84,clean,96.46,0.8747
5,cmi,92.36,noisy,93.13,0.8049
6,dafl,91.45,clean,93.20,0.7686
7,dafl,81.66,noisy,83.85,0.6521
8,adi,93.26,clean,93.45,0.8078
9,adi,90.38,noisy,92.88,0.8247


In [39]:
# noisy_df.columns
import os
if not os.path.exists('noisy/'):
    os.mkdir('noisy/')
ax= sns.barplot(data=noisy_df, x='method', y='acc@1', hue='type')
ax.set_ylim(80, 100)
ax.legend(loc='upper right')
plt.savefig('noisy/acc.pdf')

<Figure size 432x288 with 1 Axes>

In [40]:
ax = sns.barplot(data=noisy_df, x='method', y='agree@1', hue='type',palette="Set2")
ax.set_ylim(80, 100)
ax.legend(loc='upper right')
plt.savefig('noisy/agg.pdf')

<Figure size 432x288 with 1 Axes>

In [41]:
ax = sns.barplot(data=noisy_df, x='method', y='loyalty', hue='type', palette="Set2_r")
ax.set_ylim(0.6, 1)
ax.legend(loc='upper right')
plt.savefig('noisy/loyalty.pdf')

<Figure size 432x288 with 1 Axes>